In [1]:
import numpy as np
import pandas as pd
from PIL import Image
import cv2
import json

In [3]:
# Load VGG annotation file
with open('cat_dog_annotation.json', 'r') as file:
    vgg_annotation = json.load(file)

In [4]:
def vgg_to_coco(vgg_annotation):
    images = vgg_annotation['_via_img_metadata']
    
    coco_annotations = {
        'info': {},
        'licenses': [],
        'categories': [],
        'images': [],
        'annotations': []
    }
    
    category_mapping = {}
    category_id = 0
    annotation_id = 1
    
    for image_id, image_data in images.items():
        filename = image_data['filename']
        width = image_data['size']
        height = image_data['size']
        annotations = image_data['regions']
        
        # Extract the numeric part of the image ID
        image_id = int(image_id.split('.')[0])
        
        # Add image information  
        image_info = {
            'id': image_id,
            'file_name': filename,
            'width': width,
            'height': height,
        }
        coco_annotations['images'].append(image_info)
        
        for annotation in annotations:
            shape_attributes = annotation['shape_attributes']
            region_attributes = annotation['region_attributes']
            
            label = region_attributes['name']
            
            # Check if category is already defined, otherwise add it
            if label not in category_mapping:
                category_id += 1
                category_mapping[label] = category_id
                category_info = {
                    'id': category_id,
                    'name': label,
                    'supercategory': 'object',
                }
                coco_annotations['categories'].append(category_info)
            
            x = shape_attributes['x']
            y = shape_attributes['y']
            w = shape_attributes['width']
            h = shape_attributes['height']
            
            # Add annotation information
            annotation_info = {
                'id': annotation_id,
                'image_id': image_id,
                'category_id': category_mapping[label],
                'bbox': [x, y, w, h],
                'area': w * h,
                'iscrowd': 0,
            }
            coco_annotations['annotations'].append(annotation_info)
            
            annotation_id += 1
    
    return coco_annotations

In [5]:
# Convert VGG annotation to COCO format
coco_annotations = vgg_to_coco(vgg_annotation)

# Save COCO annotations to a file
with open('annotations.json', 'w') as file:
    json.dump(coco_annotations, file)
